In [ ]:
import os
from astropy.io import fits
from astropy.wcs import WCS
from astropy.modeling import models, fitting
import numpy 
from scipy import optimize
from matplotlib import pyplot
%matplotlib inline
#%matplotlib notebook
import aplpy



<h2> Open data file 

In [ ]:
#ROOTfile = "./release_galactic_skymap_fixed_head.fits"
ROOTfile = "./SgrA_galactic_uncorrelated_excess_map_fixedhead.fits"
HDUlist = fits.open(ROOTfile)
HDUlist.info()
gal_wcs = WCS(HDUlist[0].header)
print(gal_wcs)
HDUlist.close()


In [ ]:
fig_apl = aplpy.FITSFigure(ROOTfile)
#fig_apl.show_colorscale( smooth=3, vmin=-5., vmax=5.) #cmap='hot',

col_bar = fig_apl.show_colorbar()
#fig_apl.recenter(0., 0., width=3., height=2.)

#col_bar


In [ ]:
excess_data, skymap_header = fits.getdata(ROOTfile, header = True)

excess_data_N = numpy.nan_to_num(excess_data) # rm nan's for fitting 


In [ ]:

#print(skymap_header)
# G0.9+0.1
#print(excess_data_N[121:123,83:85])
#numpy.where(excess_data_N > 100)
# looks like the x/y axes are flipped, and there is a shift of 1 vs ds9


<h2> Create coordinate arrays

In [ ]:
x_bins = excess_data.shape[0]
y_bins = excess_data.shape[1]

# Create x and y indices
x = numpy.linspace(0, x_bins-1, x_bins)
y = numpy.linspace(0, y_bins-1, y_bins)
x, y = numpy.meshgrid(x, y)
coords = x, y

x_world, y_world = fig_apl.pixel2world(x,y)
coords_world = x_world, y_world

print(fig_apl.world2pixel(359.944,-0.046)) #SgrA
print(fig_apl.world2pixel(0.050,-0.150)) #J1746
print(fig_apl.world2pixel(.87,.08)) #G09


<h2> Fitting Central Gaussian


In [ ]:
SgrA_bounds = {'amplitude':(10,75), 'x_mean':(119,121), 'y_mean':(119,121), 'x_stddev':(0.25,3.), 'y_stddev':(0.25,3.), 'theta':(-1.,1.)}
SgrA_gauss = models.Gaussian2D(amplitude=30, x_mean=120, y_mean=120, x_stddev=1, y_stddev=1, theta=0, bounds=SgrA_bounds)
SgrA_bounds_2 = {'amplitude':(0.,25.), 'x_mean':(119,121), 'y_mean':(119,121), 'x_stddev':(3,10), 'y_stddev':(3,10), 'theta':(-1.,1.)}
SgrA_gauss_2 = models.Gaussian2D(amplitude=10, x_mean=120, y_mean=120, x_stddev=3, y_stddev=3, theta=0, bounds=SgrA_bounds_2)
J1746_bounds = {'amplitude':(5,15), 'x_mean':(115,117), 'y_mean':(115,117), 'x_stddev':(3,10), 'y_stddev':(3,10), 'theta':(-1.,1.)}
J1746_gauss = models.Gaussian2D(amplitude=10, x_mean=116, y_mean=116, x_stddev=3, y_stddev=3, theta=0, bounds=J1746_bounds)
sum_gauss = SgrA_gauss + SgrA_gauss_2 + J1746_gauss


In [ ]:
fit_p = fitting.LevMarLSQFitter()
fit_gauss = fit_p(sum_gauss, x, y, excess_data_N, maxiter=5000)
# try to add in sigma, or do this with curve_fit 

for i in range(0,len(fit_gauss.parameters)):
    paramline = fit_gauss.param_names[i] + ": " + str(fit_gauss.parameters[i])
    print(paramline)


In [ ]:
SgrA_double = SgrA_gauss + SgrA_gauss_2
fit_SgrA = fitting.LevMarLSQFitter()
SgrA_fit = fit_SgrA(SgrA_gauss, x, y, excess_data_N, maxiter=2500)

print(SgrA_fit.parameters)


In [ ]:
background_model = models.Box2D


In [ ]:
print(fit_p.fit_info['message'])
#print(type(fit_gauss.x_mean_0))
print(fit_gauss.x_stddev_0.value)

print(fig_apl.pixel2world(fit_gauss.x_mean_0.value,fit_gauss.y_mean_0.value))
print(fig_apl.pixel2world(fit_gauss.x_mean_1.value,fit_gauss.y_mean_1.value))

zero_pixel = fig_apl.world2pixel(0,0)
print(zero_pixel)

print(fig_apl.pixel2world(zero_pixel[0]-fit_gauss.x_stddev_1.value,zero_pixel[1]+fit_gauss.y_stddev_1.value))

#fig_ps = pyplot.figure()
#pyplot.imshow(pointsource_excess)


In [ ]:
pointsource_filename = "./central_point_excesses.fits"
pointsource_excess = fit_gauss(x,y)

if not os.path.exists(pointsource_filename):
    fits.writeto(pointsource_filename, pointsource_excess, skymap_header, clobber=True)


In [ ]:
fig_apl = aplpy.FITSFigure(pointsource_filename)
fig_apl.show_colorscale(vmin=-1,vmax=50)
#fig_apl


<h2> Excess after subtraction of central sources

In [ ]:
central_subtraction_filename = "./central_subtraction_excess.fits"
residual_skymap = excess_data - pointsource_excess 

fits.writeto(central_subtraction_filename, residual_skymap, skymap_header, clobber=True)

In [ ]:
fig_apl = aplpy.FITSFigure(central_subtraction_filename)
fig_apl.show_colorscale( smooth=3, cmap='hot' ) #cmap='hot',
col_bar = fig_apl.show_colorbar()
fig_apl.recenter(0., 0., width=3., height=2.)


<h2> Fitting G0.9+0.1


In [ ]:
G09_bounds = {'x_mean':(81,85), 'y_mean':(123,127)}
G09_gauss = models.Gaussian2D(amplitude=20,x_mean=83,y_mean=125,x_stddev=2.,y_stddev=2.,bounds=G09_bounds)

fitter_G09 = fitting.LevMarLSQFitter()
fit_G09 = fit_p(G09_gauss, x, y, excess_data_N, maxiter=1000)

print(fit_G09.parameters)


In [ ]:
G09_filename = "./G09_excesses.fits"
G09_excess = fit_gauss(x,y)

fig_G09 = pyplot.figure()
pyplot.imshow(G09_excess)

#fits.writeto(pointsource_filename, pointsource_excess, skymap_header, clobber=False)


In [ ]:
residual_G09 = residual_skymap - G09_excess


In [ ]:
fig_residual_G09 = pyplot.figure(figsize=(10,10))
fig_residual_G09.suptitle("Sgr A* Subtracted Excess Map")
ax = fig_residual_G09.add_subplot(2,1,1)
ax.set_xlabel("Galactic Longitude")
ax.set_ylabel("Galactic Latitude")
pyplot.axis([50,190,80,150])
pyplot.imshow(residual_G09)
cbar = pyplot.colorbar()
#cbar.solids.set_edgecolors('face')

ax = fig_residual_G09.add_subplot(2,1,2)
ax.set_xlabel("Galactic Longitude")
ax.set_ylabel("Galactic Latitude")
pyplot.axis([50,190,80,150])
pyplot.imshow(residual_G09)
cbar = pyplot.colorbar()
#cbar.solids.set_edgecolors('face')


<h2>FITS figure of partially subtracted map

In [ ]:
outFilename = "./subtracted_skymap_G09.fits"

fits.writeto(outFilename, residual_G09, skymap_header, clobber=True)


<h2> Other stuff..

In [ ]:
fig_apl = aplpy.FITSFigure(outFilename)
fig_apl.show_colorscale(cmap='hot', smooth=5) # vmin=0, vmax=8
col_bar = fig_apl.show_colorbar()
#col_bar


In [ ]:

p_opt_G0901, p_cov_G0901 = optimize.curve_fit(TwoD_Gaussian, coords_world, residual_skymap.ravel(), p0=init_guess_G0901)
print(p_opt_G0901)

In [ ]:
#fig_residual = pyplot.figure(figsize=(8,8))
fig_residual = pyplot.figure()
pyplot.axis([50,190,80,150])
pyplot.imshow(residual_skymap)
pyplot.Circle((120,120), 50)

cbar = pyplot.colorbar()
cbar.solids.set_edgecolors('face')

#fig_residual.set_axes


In [ ]:
# print(x)
# print(x_world)
# print(x_world[2,6])
# print(y_world[239,6])

print(fig_apl.pixel2world(100,180))
print(fig_apl.pixel2world(101,180))

#excess_data_N[81:85,123:127]
#xworld = fig_apl.pixel2world()

models.Gaussian2D?


fermi tools
maximum likelihood
Wilke's theorem 